In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import re
import os
from matplotlib.colors import ListedColormap
from skimage.measure import label, regionprops
from skimage.filters import threshold_otsu
from skimage.morphology import disk, binary_opening, binary_closing, octagon, square
from skimage.morphology import remove_small_objects
import scipy.io
from tqdm import tqdm
import json
import openpyxl
import sys
import inspect
from skimage import exposure
from PIL import Image
from scipy.spatial.distance import pdist

import math


In [2]:
def usFOV(img,perc_header=0.075):
    img = np.array(img)

    # Calculate the new position of y.
    new_y0 = int(np.round(img.shape[0] * perc_header))

    # Convert to grayscale and binarize
    gray = cv2.cvtColor(img[new_y0-1:, :, :], cv2.COLOR_RGB2GRAY)
    binary = gray > 0

    # Label the regions and keep only the largest one.
    labeled_img = label(binary)
    largest_region = max(regionprops(labeled_img), key=lambda region: region.area)
    binary = labeled_img == largest_region.label

    # Calculate the size of the structuring element.
    tsl = disk(round(min(binary.shape) * 0.09))

    # Apply the morphological opening operation.
    box1 = binary_opening(binary, tsl)

    # Label the regions again.
    box1 = label(box1)

    # Get the coordinates of the bounding box.
    box1_props = max(regionprops(box1), key=lambda region: region.area)
    box1_bbox = box1_props.bbox

    # Apply another morphological opening operation with an octagon.
    box2 = binary_opening(binary, octagon(9, 1))
    box2 = label(box2)
    box2_props = max(regionprops(box2), key=lambda region: region.area)
    box2_bbox = box2_props.bbox

    # Calculate the coordinates of the region of interest (ROI).
    roi = (box2_bbox[1], box1_bbox[0], box2_bbox[3] - box2_bbox[1], box1_bbox[2] - box1_bbox[0])
    roi = tuple(map(int, roi))

    # Crop the region of interest (ROI) from the original image.
    fov = img[new_y0:, :, :]
    fov = fov[roi[1]:roi[1] + roi[3], roi[0]:roi[0] + roi[2], :]
    
    return fov

In [6]:
def usDcallipers(impath):
    
    if impath.endswith(".bmp"):
        factor=15
    elif impath.endswith(".jpg") or impath.endswith(".jpeg"):
        factor=45
        
    f=cv2.imread(impath)

    # Difference between red and blue channels and normalization
    imagen=usFOV(f)
    mg1 = np.abs(imagen[:,:,0] - imagen[:,:,2])
    mg1 = (mg1 - np.min(mg1)) / (np.max(mg1) - np.min(mg1))

    # Thresholding with a high value to identify only crosses and ignore other artifacts
    cm_init = mg1 > 0.55
    
    # Opening and closing to have a better approximation of a cross shape
    se1 = np.ones((1, 2))
    se2 = np.ones((9, 9))

    cm_init = remove_small_objects(cm_init, factor)

    # Perform the morphological opening operation.
    cm = binary_opening(cm_init, se1)

    # Perform the morphological closing operation.
    cm = binary_closing(cm,se2)
    h, w = cm.shape

    # Remove artifacts in defined regions to prevent erroneous calliper detection 
    cm[0:int(h*0.25), int(w*(1-0.15)):] = 0  # Region 1
    cm[int(h*(0.9)):, :] = 0                # Region 2
    cm[:, 0:int(w*0.10)] = 0   

    cm = remove_small_objects(cm, 3)
    cm= cm.astype(np.uint8)
    labeled_cm = label(cm)

    if np.max(labeled_cm) > 2:
        # Filter to keep only the three largest connected components.
        cm = remove_small_objects(cm, 3)
        labeled_cm = label(cm)

        # Obtain the properties of the remaining regions.
        regions = regionprops(labeled_cm)

        # Initialize a list to store the centroids of the remaining regions.
        centroids = []

        # Iterate over the regions and obtain the centroids.
        for region in regions:
            centroids.append(region.centroid)

        # Convert the list of centroids into a NumPy array.
        centroids = np.array(centroids)

        # Calculate the distances between the centroids.
        percs = pdist(centroids) / np.sqrt(w ** 2 + h ** 2)
        cond = np.any((percs[percs > 0.02]) < 0.07)

    # Obtaining calliper coordinates
    labeled_cm = label(cm)

    p1=[0,0]
    p2=[0,0]

    # Labeled regions.
    regions = regionprops(labeled_cm)
    prueba=len(np.unique(labeled_cm))
    
    # Find the coordinates of the labeled regions.
    for region in regions:
        if region.label == prueba-1:
            y1, x1 = region.coords[:, 0], region.coords[:, 1]
            p1 = [np.mean(x1), np.mean(y1)]
        elif region.label == prueba-2:
            y2, x2 = region.coords[:, 0], region.coords[:, 1]
            p2 = [np.mean(x2), np.mean(y2)]

   # p1 always above p2.
    if p2[1] - p1[1] < 0:
        p1, p2 = p2, p1

    return cm, p1, p2,imagen

In [14]:
def usPreprocess(f):
    
    cm, p1, p2,f1 = usDcallipers(f)
    gray = cv2.cvtColor(f1, cv2.COLOR_RGB2GRAY)
    _, bm = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)
    
    # Background
    bm = fillHoles(bm)
    bm = cv2.erode(bm, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3)))
    bm = cv2.morphologyEx(bm, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7)))
    
    # Normalization, G and B channels.
    mg2 = np.abs(f1[:,:,0] - f1[:,:,1])
    mg2 = (mg2 - np.min(mg2)) / (np.max(mg2) - np.min(mg2))
    
    am_init = mg2 > 0.1
    dam_init = cv2.dilate(am_init.astype(np.uint8), np.ones((2, 2), np.uint8))
    
    if cm.shape != dam_init.shape:
        dam_init = cv2.resize(dam_init, (cm.shape[1], cm.shape[0]))
    
    # Perform the logical OR operation.
    am = np.logical_or(cm, dam_init)
    
    # Convert image to a single channel.
    rimg = np.mean(f1[:, :, 0:2], axis=2).astype(np.uint8)
    
    # Create a dilated region mask.
    dregion_mask = cv2.dilate(am_init.astype(np.uint8), np.ones((2, 2), np.uint8))
    
    # Inpaint the image.
    g = cv2.inpaint(rimg, dregion_mask, 3, cv2.INPAINT_TELEA)

    return g, am, bm, p1, p2

def fillHoles(bm):
    # Find the contours in the binary image.
    contours, _ = cv2.findContours(bm, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw the contours on a black image.
    filled = np.zeros_like(bm)
    cv2.drawContours(filled, contours, -1, 255, thickness=cv2.FILLED)

    return filled

In [ ]:
def usSeg(cm, p1, p2):
    # Create a copy of the segmentation image
    seg = np.zeros_like(cm)

    p1 = (int(p1[0]), int(p1[1]))
    p2 = (int(p2[0]), int(p2[1]))
    angle = 160

    angle_rad = math.radians(angle)
    a = 120

    p1_up = (int(p1[0] + a * math.cos(angle_rad)), int(p1[1] - a * math.sin(angle_rad))) 
    p1_down = (int(p1[0] - (a-50) * math.cos(angle_rad)), int(p1[1] + (a-50) * math.sin(angle_rad)))

    # Calculate the control points for the Bézier curve between p1_up and p2_up
    p2_up = (p2[0], p2[1] - 50)  # Adjust p2_up to make it a higher point
    p1_up = (p1[0], p1[1] - 50)  # Adjust p1_up to make it a higher point
    
    # Calculate the control point for the curve (you can adjust this value)
    control_point = (int((p1_up[0] + p2_up[0]) / 2), int((p1_up[1] + p2_up[1]) / 2) - 30)

    # Generate the Bézier curve
    curve_points = bezier_curve(p2_up, control_point, p1_up, num_points=50)

    # Create the polygon
    polygon_points = np.array([p1_up, p2_up] + curve_points.tolist() + [p1_down, p2_down], dtype=np.int32)

    # Fill the polygon in the segmentation image
    cv2.fillPoly(seg, [polygon_points], color=1)

    # Apply the segmentation to the image with additional rows (using cm with extra rows)
    seg = seg * (cm == 255).astype(np.uint8)

    # Define the kernel size for erosion
    kernel = np.ones((5, 5), np.uint8)

    # Create a mask that only covers the region between p1_up, p2_up, and up to p2_down
    mask = np.zeros_like(seg)

    # Draw the polygon bounded by the points
    polygon_points = np.array([p1_up, p2_up, p2_down], dtype=np.int32)
    cv2.fillPoly(mask, [polygon_points], color=1)

    # Apply erosion only to the selected region
    eroded = cv2.erode(seg, kernel)
    
    # Apply erosion to the selected region in the mask
    seg[mask == 1] = eroded[mask == 1]

    return seg